# Lab 5 : Exploratory Data Analysis

## Use Huggingface to download the dataset 🤗.

In [1]:
!hf download celesca/election_results_2569 --repo-type dataset --local-dir ./

Fetching 3 files: 100%|███████████████████████████| 3/3 [00:02<00:00,  1.27it/s]
Download complete: 100%|██████████████████████| 818k/818k [00:02<00:00, 347kB/s]/Users/beam/Workspace/Course/my-cpe-lab/Y2/CPE232/sessions/05_eda
Download complete: 100%|██████████████████████| 818k/818k [00:02<00:00, 343kB/s]


In [2]:
!unzip -q ./data.zip

checkdir error:  cannot create /Users/beam/Workspace/Course/my-cpe-lab/Y2/CPE232/sessions/05_eda/all_constituency/�+����--ʦ�+
                 Illegal byte sequence
                 unable to process all_constituency/�+����--ʦ�+/.
checkdir error:  cannot create /Users/beam/Workspace/Course/my-cpe-lab/Y2/CPE232/sessions/05_eda/all_constituency/�+����--ʦ�+
                 Illegal byte sequence
                 unable to process all_constituency/�+����--ʦ�+/�+����--ʦ�+ ��� 1/.
checkdir error:  cannot create /Users/beam/Workspace/Course/my-cpe-lab/Y2/CPE232/sessions/05_eda/all_constituency/�+����--ʦ�+
                 Illegal byte sequence
                 unable to process all_constituency/�+����--ʦ�+/�+����--ʦ�+ ��� 1/constituency_candidates.csv.
checkdir error:  cannot create /Users/beam/Workspace/Course/my-cpe-lab/Y2/CPE232/sessions/05_eda/all_constituency/�+����--ʦ�+
                 Illegal byte sequence
                 unable to process all_constituency/�+����--ʦ�+/�+����--ʦ�+ ���

In [3]:
import shutil
from pathlib import Path

# Create data directory
data_dir = Path("data")
data_dir.mkdir(exist_ok=True)

# Extract only CSV files to data directory
data_folders = ["all_constituency", "area", "party", "partylist", "referendum", "seat", "winner"]

for folder in data_folders:
    folder_path = Path(folder)
    if folder_path.exists():
        for csv_file in folder_path.glob("*.csv"):
            shutil.copy(csv_file, data_dir / csv_file.name)
            print(f"Copied {csv_file.name} to data/")


Copied areas.csv to data/
Copied partyStandings.csv to data/
Copied party_list_top2.csv to data/
Copied map.csv to data/
Copied seat500.csv to data/
Copied winner.csv to data/


In [4]:
# Clean up: Remove the original data folders after extraction
for folder in data_folders:
    folder_path = Path(folder)
    if folder_path.exists():
        shutil.rmtree(folder_path)
        print(f"Removed {folder} directory")

# Also remove the zip file if no longer needed
if Path("data.zip").exists():
    Path("data.zip").unlink()
    print("Removed data.zip")

print("\nCSV files are now in the 'data/' directory")


Removed all_constituency directory
Removed area directory
Removed party directory
Removed partylist directory
Removed referendum directory
Removed seat directory
Removed winner directory
Removed data.zip

CSV files are now in the 'data/' directory


## Thai Font in Matplotlib

Download Thai font from the link below and place it in your working directory:
https://drive.google.com/drive/folders/1s_xqY9Wyx9QNNPZkZlTbFlNM5-zQc4iQ

In [ ]:
# Set the path to your Thai font file
# Download from: https://drive.google.com/drive/folders/1s_xqY9Wyx9QNNPZkZlTbFlNM5-zQc4iQ
font_path = "THSarabunNew.ttf"  # Place the font file in your working directory

In [ ]:
# Test Thai Font
import matplotlib.pyplot as plt
from matplotlib import font_manager, rcParams

font_path = "THSarabunNew.ttf"
font_manager.fontManager.addfont(font_path)
rcParams["font.family"] = "TH Sarabun New"